# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [72]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [32]:
X_tr = pd.read_excel("/content/X_train.xlsx", index_col=[1])
X_te = pd.read_excel("/content/X_test.xlsx", index_col=[1])

In [42]:
y_tr = pd.read_excel("/content/y_train.xlsx").drop(columns=["Unnamed: 0"])
y_te = pd.read_excel("/content/y_test.xlsx").drop(columns=["Unnamed: 0"])

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [43]:
X_train, X_validation, y_train, y_validation = train_test_split(X_tr, y_tr, test_size=0.25, random_state=42)

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [44]:
tree = DecisionTreeClassifier(max_depth=4, random_state=42, min_samples_leaf=20).fit(X_train, y_train)

In [45]:
indices = tree.feature_importances_.argsort()[-3:][::-1]

In [46]:
best_variables = X_train.columns[indices]
best_variables

# As 3 melhores variáveis são: tGravityAcc-min()-X; fBodyAccJerk-bandsEnergy()-1,8 e tGravityAcc-mean()-Y

Index(['tGravityAcc-min()-X', 'fBodyAcc-mad()-X', 'tGravityAcc-mean()-Y'], dtype='object')

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [47]:
%%time
improved_X_train = X_train[best_variables]

tree = DecisionTreeClassifier(max_depth=4, random_state=42, min_samples_leaf=20).cost_complexity_pruning_path(improved_X_train, y_train)

CPU times: user 17.3 ms, sys: 1 ms, total: 18.3 ms
Wall time: 18.2 ms


In [48]:
ccp_alphas, impurities = tree.ccp_alphas, tree.impurities

In [49]:
accuracy_by_ccp = []

In [50]:
for ccp_alpha in ccp_alphas:
  tree = DecisionTreeClassifier(min_samples_leaf=20, random_state=42, ccp_alpha=ccp_alpha).fit(improved_X_train, y_train)
  y_pred = tree.predict(X_validation[best_variables])
  accuracy = accuracy_score(y_validation, y_pred)

  accuracy_by_ccp.append({
      "ccp_alpha": ccp_alpha,
      "accuracy": accuracy
  })

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [52]:
dataframe = pd.DataFrame(accuracy_by_ccp)

In [55]:
dataframe.loc[dataframe["accuracy"].idxmax()]

# O melhor ccp_alpha encontrado foi 0.000000

ccp_alpha    0.000000
accuracy     0.890642
Name: 0, dtype: float64

In [66]:
tree = DecisionTreeClassifier(min_samples_leaf=20, random_state=42, ccp_alpha=0.000000).fit(improved_X_train, y_train)

In [68]:
y_pred = tree.predict(X_validation[best_variables])
accuracy = accuracy_score(y_validation, y_pred)

In [70]:
# Acurácia do modelo
accuracy

0.8906420021762785

In [73]:
conf_matrix = confusion_matrix(y_validation, y_pred)

In [74]:
conf_matrix

array([[271,  26,   6,   0,   0,   0],
       [ 36, 206,  14,   0,   0,   0],
       [ 41,  16, 199,   0,   0,   0],
       [  0,   1,   0, 287,  40,   0],
       [  0,   0,   0,  21, 331,   0],
       [  0,   0,   0,   0,   0, 343]])

In [75]:
import plotly.graph_objs as go

In [76]:
heatmap = go.Heatmap(
  z=conf_matrix,
  x=['Predito: ' + str(i) for i in range(conf_matrix.shape[1])],
  y=['Real: ' + str(i) for i in range(conf_matrix.shape[0])],
  colorscale='Viridis'
)


layout = go.Layout(
    title='Matriz de Confusão',
    xaxis=dict(title='Predito'),
    yaxis=dict(title='Real')
)


fig = go.Figure(data=[heatmap], layout=layout)

In [78]:
fig.show()